# Основные показатели каналов
Пример расчета базовых показателей по каналам

## Описание задачи и условий расчета

- Период: 03.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 18+
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index										
- Статистики: Rtg%, Share, AvReach%, ATVReach

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['возраст'])

Запрошены записи: 0 - 125
Всего найдено записей: 125



,id,valueId,name,colName,valueName
0,2,33,Возраст,Age,33
1,16,1,Возраст детей: До 1 года,KidsAge2,Да
2,16,0,Возраст детей: До 1 года,KidsAge2,Нет
3,20,0,Возраст детей: 7-11 лет,KidsAge6,Нет
4,20,1,Возраст детей: 7-11 лет,KidsAge6,Да
...,...,...,...,...,...
120,2,10,Возраст,Age,10
121,2,89,Возраст,Age,89
122,2,8,Возраст,Age,8
123,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


## Формирование задания
Зададим условия расчета

In [3]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-08-03', '2022-08-03')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = 'age >= 18'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter=None

# Указываем список статистик для расчета
statistics = ['RtgPer', 'Share', 'AvReachPer', 'ATVReach']

# Указываем срезы - разбиваем на каналы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

## Расчет задания

In [4]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_task('timeband', date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))
df

Расчет задачи (id: 08425612-60da-492d-8caa-b1b548acb8c6) [= = = ] время расчета: 0:00:11.085136


,tvNetName,Share,ATVReach,RtgPer,AvReachPer
0,ТВ ЦЕНТР,2.676485,49.955924,0.386905,11.152706
1,РЕН ТВ,6.098150,72.120811,0.881532,17.601107
2,МИР,2.273100,65.001431,0.328593,7.279443
3,ИЗМЕРЯЕМОЕ ТЕМАТИЧЕСКОЕ ТВ,16.432560,107.572939,2.375446,31.798358
4,Ю,0.687687,72.402000,0.099410,1.977163
5,ДОМ КИНО,0.917429,57.809080,0.132621,3.303534
6,РОССИЯ К,0.653866,21.257057,0.094521,6.403066
7,ДОМАШНИЙ,4.486212,93.693507,0.648515,9.967190
8,КАРУСЕЛЬ,2.623387,81.250566,0.379230,6.721070
9,СТС LOVE,0.279623,40.649241,0.040422,1.431935


## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [5]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('01_basic_channels_figures'))
df_info = mtask.task_builder.get_report_info()
df.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()